In [1]:
import numpy as np
import math
import xarray as xr
import dask
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import netCDF4
import pandas as pd
from sklearn.cluster import KMeans
from scipy.ndimage.filters import uniform_filter1d

In [10]:
path = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-[0124]*'
real_ds = xr.open_mfdataset(path)
Q_CRM = real_ds.CRM_QV_LON_0e_to_360e_LAT_20s_to_20n
W_CRM = real_ds.CRM_W_LON_0e_to_360e_LAT_20s_to_20n
T_CRM = real_ds.CRM_T_LON_0e_to_360e_LAT_20s_to_20n
Precip_CRM = real_ds.CRM_PREC_LON_0e_to_360e_LAT_20s_to_20n

Q_CRM = xr.DataArray.squeeze(Q_CRM).values
T_CRM = xr.DataArray.squeeze(T_CRM).values
TwoD_W_CRM = xr.DataArray.squeeze(W_CRM).values
Scalar_W_CRM = xr.DataArray.squeeze(W_CRM[:,18,:,:,:]).values
Precip_CRM = xr.DataArray.squeeze(Precip_CRM).values

In [12]:
print(Precip_CRM.shape)

(64, 128, 22, 144)


In [ ]:
others = netCDF4.Dataset("/fast/gmooers/Raw_Data/extras/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-01-72000.nc")
plev = np.array(others.variables['lev'])
ps = np.array(others.variables['PS'])
g = 9.81 #m/s^2
hyai = np.array(others.variables['hyai'])
hybi = np.array(others.variables['hybi'])
#print(hyai.shape)
#print(hyai)
cp = 1004.0
L = 2256000.0
PS = 1e5
P0 = 1e5
P = P0*hyai+PS*hybi # Total pressure [Pa]
dp = P[1:]-P[:-1] # Differential pressure [Pa]
Scalar_Q_CRM = np.nansum(Q_CRM*dp[None, :, None, None, None]*L/g, axis=1)
Scalar_T_CRM = np.nansum(T_CRM*dp[None, :, None, None, None]*cp/g, axis=1)

In [ ]:
Q_Reshape = np.zeros(shape=(len(Scalar_Q_CRM)*len(Scalar_Q_CRM[0][0])*len(Scalar_Q_CRM[0][0][0]),128))
Q_Reshape[:,:] = np.nan
T_Reshape = np.zeros(shape=(len(Scalar_T_CRM)*len(Scalar_T_CRM[0][0])*len(Scalar_T_CRM[0][0][0]),128))
T_Reshape[:,:] = np.nan
W_Reshape = np.zeros(shape=(len(Scalar_W_CRM)*len(Scalar_W_CRM[0][0])*len(Scalar_W_CRM[0][0][0]),128))
W_Reshape[:,:] = np.nan
Precip_Reshape = np.zeros(shape=(len(Precip_CRM)*len(Precip_CRM[0][0])*len(Precip_CRM[0][0][0]),128))
Precip_Reshape[:,:] = np.nan

In [13]:
print(TwoD_W_CRM.shape)

(64, 30, 128, 22, 144)


In [ ]:
TwoD_W_Reshape = np.zeros(shape=(len(TwoD_W_CRM)*len(TwoD_W_CRM[0][0][0])*len(TwoD_W_CRM[0][0][0][0]),30,128))
TwoD_W_Reshape[:,:,:] = np.nan

In [ ]:
count = 0
for i in range(len(Scalar_Q_CRM)):
    for j in range(len(Scalar_Q_CRM[0][0])):
        for k in range(len(Scalar_Q_CRM[0][0][0])):
            Q_Reshape[count,:] = Scalar_Q_CRM[i,:,j,k]
            T_Reshape[count,:] = Scalar_T_CRM[i,:,j,k]
            W_Reshape[count,:] = Scalar_W_CRM[i,:,j,k]
            Precip_Reshape[count,:] = Precip_CRM[i,:,j,k]
            count = count+1

In [ ]:
count = 0
for i in range(len(TwoD_W_CRM)):
    print(i)
    for j in range(len(TwoD_W_CRM[0][0][0])):
        for k in range(len(TwoD_W_CRM[0][0][0][0])):
            TwoD_W_Reshape[count,:,:] = TwoD_W_CRM[i,:,:,j,k]
            count = count+1

In [ ]:
q_max = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Max_Scalar.npy')
q_min = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Min_Scalar.npy')
t_max = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Max_Scalar.npy')
t_min = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Min_Scalar.npy')
precip_max = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Max_Scalar.npy')
precip_min = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Min_Scalar.npy')
w_max = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Max_Scalar.npy')
w_min = np.load('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Min_Scalar.npy')

In [ ]:
Q_rescaled_test = np.interp(Q_Reshape, (q_min, q_max), (0, +1))

T_rescaled_test = np.interp(T_Reshape, (t_min, t_max), (0, +1))

W_rescaled_test = np.interp(W_Reshape, (w_min, w_max), (0, +1))

Precip_rescaled_test = np.interp(Precip_Reshape, (precip_min, precip_max), (0, +1))

In [ ]:
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Q_Test.npy', Q_rescaled_test)

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_T_Test.npy', T_rescaled_test)

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_W_Test.npy', W_rescaled_test)
np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_2D_W_Test.npy', TwoD_W_Reshape)

np.save('/fast/gmooers/Preprocessed_Data/CRM_Scalar/Space_Time_Precip_Test.npy', Precip_rescaled_test)